In [1]:
from HEEM_VQE import *
from qiskit.providers.ibmq.runtime import UserMessenger
from qiskit import Aer
import matplotlib.pyplot as plt

In [2]:
meta = {
  "name": "HEEM_VQE",
  "description": "Variationa Quantum Eigensolver with hardware efficient entangled measurements",
  "max_execution_time": 18000,
  "version": "1.0",
}

In [3]:
meta["parameters"] = [
    {"name": "iters", "description": "number of iterations, default=10", "type": "int", "required": False },
    {"name": "pars", "description": "Initial vector of parameters.", "type": "ndarray", "required": False}, 
    {"name": "shots", "description": "Number of shots to take per circuit, default=8192", "type": "int", "required": False},
    {"name": "grouping", "description": "Method for grouping the local observables of the Hamiltonian into compatible measurements: tensor product basis ('TPB') or entangled measurements ('Entangled') , default='TPB'.", "type": "str", "required": False},
    {"name": "conectivity", "description": "Device conectivity.", "type": "list", "required": False}
  ]

meta['return_values'] = [
    {"name": "result", "description": "Final result in a dictionary.", "type": "dict"}
  ]

meta["interim_results"] = [
    {"name": "params", "description": "Energy evaluations", "type": "ndarray"},
  ]


In [4]:
from qiskit import IBMQ
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-csic', group='internal', project='iff-csic') 

In [5]:
program_id = provider.runtime.upload_program(data='HEEM_VQE.py', metadata=meta)
program_id

RuntimeDuplicateProgramError: 'Program with the same name already exists.'

In [ ]:
# program_id = 'heem-vqe'
# provider.runtime.delete_program(program_id)

In [9]:
program_id = 'heem-vqe'

In [10]:
prog = provider.runtime.program(program_id)
print(prog)

heem-vqe:
  Name: HEEM_VQE
  Description: Variationa Quantum Eigensolver with hardware efficient entangled measurements
  Version: 1.0
  Creation date: 2021-10-13T14:29:48.000000
  Max execution time: 18000
  Input parameters:
    - iters:
      Description: number of iterations, default=10
      Type: int
      Required: False
    - pars:
      Description: Initial vector of parameters.
      Type: ndarray
      Required: False
    - shots:
      Description: Number of shots to take per circuit, default=8192
      Type: int
      Required: False
    - grouping:
      Description: Method for grouping the local observables of the Hamiltonian into compatible measurements: tensor product basis ('TPB') or entangled measurements ('Entangled') , default='TPB'.
      Type: str
      Required: False
    - conectivity:
      Description: Device conectivity.
      Type: list
      Required: False
  Interim results:
    - params:
      Description: Energy evaluations
      Type: ndarray
  Returns

In [11]:
inputs = { "iters": 5 ,
           "grouping": "Entangled",
           "shots" : 2**13,
           "conectivity" : False }
# backend = provider.backend.ibmq_qasm_simulator
backend = provider.get_backend('ibmq_quito')
options = {'backend_name': backend.name()}

In [12]:
get_backend_conectivity(backend)

[(0, 1), (1, 0), (1, 2), (1, 3), (2, 1), (3, 1), (3, 4), (4, 3)]

In [13]:
interm_results = []
def vqe_callback(job_id, data):
    interm_results.append(data)

In [14]:
job = provider.runtime.run(program_id, options=options, inputs=inputs, callback=vqe_callback)

In [15]:
job.result()

RuntimeJobFailureError: 'Unable to retrieve job result. Job c5jf4rl7js99mk9ljhk0 ran longer than maximum execution time. Job was cancelled:\n2021-10-13T10:52:07.031148964-04:00 Couldn\'t find cython becke routine\n2021-10-13T10:52:07.031271195-04:00 /usr/local/lib/python3.8/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than \'r\' is deprecated. Pass the mode to h5py.File() instead.\n2021-10-13T10:52:07.031271195-04:00   h5py.get_config().default_file_mode = \'a\'\n2021-10-13T10:52:07.377075800-04:00 /usr/local/lib/python3.8/site-packages/qiskit/utils/run_circuits.py:612: UserWarning: max_credits is not a recognized runtime option and may be ignored by the backend.\n2021-10-13T10:52:07.377075800-04:00   job = _run_circuits_on_backend(\n2021-10-13T10:52:46.259401613-04:00 /usr/local/lib/python3.8/site-packages/nats/aio/client.py:888: DeprecationWarning: The loop argument is deprecated since Python 3.8, and scheduled for removal in Python 3.10.\

In [ ]:
energies= [ idx['energy'] for idx in interm_results ]

In [ ]:
plt.plot(energies)
plt.grid()

In [ ]:
plt.plot(energies)
plt.grid()